# Predicting Price with Size, Location, and Neighborhood

In [15]:
pwd

'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model'

## Library Importing

In [16]:
from glob import glob
import pandas as pd
import seaborn as sns #for data visualisation
from category_encoders import OneHotEncoder  #encoding categorical values
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact #for the interactive dashboard
from sklearn.impute import SimpleImputer #3imputting missing values
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401--automaed quality control code.
from sklearn.metrics import mean_absolute_error #to evaluate the model
from sklearn.pipeline import make_pipeline  #make the pipoeline
from sklearn.utils.validation import check_is_fitted #for unit testing

## Data Wrangling

In [17]:
#Loading the dataset with a function
def wrangle (filepath):
    #Load the CSV file into file path-local function applied
    df = pd.read_csv(filepath, encoding='ISO-8859-1') #read csv to the dataframe 
    #To display 'Capital Federal' only 
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    #To display 'Apartment' only 
    mask_apt = df["property_type"] == "apartment" 
    #To display price below'400,000'
    mask_price = df["price_aprox_usd"] < 400_000
    #Subset
    df = df[ mask_ba & mask_apt & mask_price]
    
    #Remove outliers by 'surface_area_m2' -----removing the 10th and 90th quantiles
    low = df["surface_covered_in_m2"].quantile(0.1)
    high = df["surface_covered_in_m2"].quantile(0.9)
    
    mask_area = df["surface_covered_in_m2"].between(low, high)
    
    df = df[mask_area]
    
    #split the lat-lon column
    df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
    df.drop(columns="lat-lon", inplace=True)
    
    #split place_with_parent_names
    df["neighborhood"] = df["place_with_parent_names"].str.split("|", expand=True)[3]
    df.drop(columns="place_with_parent_names", inplace=True)
    
    #drop features with high  null counts
    df.drop(columns=["floor","expenses"], inplace = True) 
    
    print('low = ', low, ';high = ', high)
    print('df.shape = ', df.shape)
    
    return df

In [18]:
#Collecting the files with the glob function
files = glob("C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model/buenos-aires-real-estate-*.csv")
files

['C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\buenos-aires-real-estate-1.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\buenos-aires-real-estate-2.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\buenos-aires-real-estate-3.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\buenos-aires-real-estate-4.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\buenos-aires-real-estate-5.csv']

In [19]:
#Loading the files with list comprehension to a df 
frames = [wrangle(file) for file in files]

low =  31.0 ;high =  100.0
df.shape =  (1343, 16)
low =  31.0 ;high =  100.0
df.shape =  (1315, 16)
low =  31.0 ;high =  101.70000000000005
df.shape =  (1288, 16)
low =  30.0 ;high =  98.0
df.shape =  (1305, 16)
low =  30.0 ;high =  100.0
df.shape =  (1331, 16)


In [20]:
type(frames[0]) #indicating its a list 

pandas.core.frame.DataFrame

In [21]:
frames[0].head() #dataframe 1

,Unnamed: 0,operation,property_type,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms,properati_url,lat,lon,neighborhood
4,5,sell,apartment,129000.0,USD,1955949.6,129000.0,76.0,70.0,1697.368421,1842.857143,NaN,http://chacarita.properati.com.ar/10qlv_venta_...,-34.584651,-58.454693,Chacarita
9,10,sell,apartment,87000.0,USD,1319128.8,87000.0,48.0,42.0,1812.500000,2071.428571,NaN,http://villa-luro.properati.com.ar/12m82_venta...,-34.638979,-58.500115,Villa Luro
29,30,sell,apartment,118000.0,USD,1789163.2,118000.0,NaN,54.0,NaN,2185.185185,2.0,http://caballito.properati.com.ar/11wqh_venta_...,-34.615847,-58.459957,Caballito
40,41,sell,apartment,57000.0,USD,864256.8,57000.0,42.0,42.0,1357.142857,1357.142857,2.0,http://constitucion.properati.com.ar/k2f0_vent...,-34.625222,-58.382382,Constitución
41,42,sell,apartment,90000.0,USD,1364616.0,90000.0,57.0,50.0,1578.947368,1800.000000,3.0,http://once.properati.com.ar/suwa_venta_depart...,-34.610610,-58.412511,Once


In [22]:
#concatenate the dataframes 
df = pd.concat(frames, ignore_index = True)
df.head()

,Unnamed: 0,operation,property_type,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms,properati_url,lat,lon,neighborhood
0,5,sell,apartment,129000.0,USD,1955949.6,129000.0,76.0,70.0,1697.368421,1842.857143,NaN,http://chacarita.properati.com.ar/10qlv_venta_...,-34.584651,-58.454693,Chacarita
1,10,sell,apartment,87000.0,USD,1319128.8,87000.0,48.0,42.0,1812.500000,2071.428571,NaN,http://villa-luro.properati.com.ar/12m82_venta...,-34.638979,-58.500115,Villa Luro
2,30,sell,apartment,118000.0,USD,1789163.2,118000.0,NaN,54.0,NaN,2185.185185,2.0,http://caballito.properati.com.ar/11wqh_venta_...,-34.615847,-58.459957,Caballito
3,41,sell,apartment,57000.0,USD,864256.8,57000.0,42.0,42.0,1357.142857,1357.142857,2.0,http://constitucion.properati.com.ar/k2f0_vent...,-34.625222,-58.382382,Constitución
4,42,sell,apartment,90000.0,USD,1364616.0,90000.0,57.0,50.0,1578.947368,1800.000000,3.0,http://once.properati.com.ar/suwa_venta_depart...,-34.610610,-58.412511,Once


In [23]:
 df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6582 entries, 0 to 6581
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  6582 non-null   int64  
 1   operation                   6582 non-null   object 
 2   property_type               6582 non-null   object 
 3   price                       6582 non-null   float64
 4   currency                    6582 non-null   object 
 5   price_aprox_local_currency  6582 non-null   float64
 6   price_aprox_usd             6582 non-null   float64
 7   surface_total_in_m2         4752 non-null   float64
 8   surface_covered_in_m2       6582 non-null   float64
 9   price_usd_per_m2            4536 non-null   float64
 10  price_per_m2                6582 non-null   float64
 11  rooms                       5286 non-null   float64
 12  properati_url               6582 non-null   object 
 13  lat                         6316 

In [24]:
#shows percentage of representation of missing values more than 50% drop
#sum of null values in each column 0 rep false and 1 is true
df.isnull().sum()/len(df)

Unnamed: 0                    0.000000
operation                     0.000000
property_type                 0.000000
price                         0.000000
currency                      0.000000
price_aprox_local_currency    0.000000
price_aprox_usd               0.000000
surface_total_in_m2           0.278031
surface_covered_in_m2         0.000000
price_usd_per_m2              0.310848
price_per_m2                  0.000000
rooms                         0.196901
properati_url                 0.000000
lat                           0.040413
lon                           0.040413
neighborhood                  0.000000
dtype: float64

### Next